In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import ensemble

In [24]:
def print_score(model, x, y):
    print('Score:        {:.4f}%'.format(model.score(x, y) * 100))
    print('Kaggle Score: {:.0f}'.format(np.sqrt(np.mean((model.predict(x) - y)**2))))    

In [25]:
# Cleaning Data

df = pd.read_csv('data/stock_XY_train.csv')
print('OG Data Size:{}'.format(df.shape))


dropped_columns = (df[df.columns[df.isnull().mean() > 0.15]].columns)
df = df[df.columns[df.isnull().mean() < 0.15]] # TO-DO: Tinker around with mean threshold.
df = df.dropna()
print('New Data Size:{}'.format(df.shape))
del df['operatingProfitMargin'] # Got rid of this column because it is all `1`. No reason to keep.

del df['Ticker']
del df['Sector']
del df['Yr']

OG Data Size:(12379, 226)
New Data Size:(8977, 156)


In [26]:
# Separating to train and test data
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

# Grabbing stats in order to normalize data
train_stats = train_df.describe()
train_stats.pop('Buy')
train_stats = train_stats.transpose()

# Separating labels
train_label = train_df.pop('Buy')
test_label = test_df.pop('Buy')

# Normalizing Data
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_df)
normed_test_data = norm(test_df)

In [ ]:
params = {
    'n_estimators': [100],
    'max_depth': [5, 8],
    'learning_rate': [0.05, 0.075,],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
}

gradient_boosting_regressor = GridSearchCV(
    ensemble.GradientBoostingRegressor(),
    params,
)


gradient_boosting_regressor.fit(normed_train_data, train_label)

In [38]:
print(gradient_boosting_regressor.score(normed_train_data, train_label))
print(gradient_boosting_regressor.best_params_)

0.40920939634251746
{'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100}


In [19]:
columns = df.loc[:, df.columns != 'Buy'].columns
columns

# x_train, x_test, y_train, y_test = train_test_split(normed_train_data[columns], df['Buy'], test_size=0.4, random_state=42)

gb = ensemble.GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.05, criterion='mse')
gb.fit(normed_train_data, train_label)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [40]:
print_score(gradient_boosting_regressor, normed_test_data, test_label)

Score:        20.5369%
Kaggle Score: 0
